In [1]:
# AWS Bedrock Agent Sample
# This notebook demonstrates how to call a Bedrock agent with a given prompt

import boto3
import json
from typing import Dict, Any
import time


# AWS Bedrock Agent Integration

This notebook demonstrates how to:
- Set up AWS Bedrock client
- Call a Bedrock agent with prompts
- Handle agent responses
- Parse and display results

## Prerequisites
- AWS credentials configured (via AWS CLI, IAM role, or environment variables)
- Bedrock agent created and deployed
- Appropriate IAM permissions for Bedrock access


In [14]:
# Configuration - Update these values with your Bedrock agent details
AGENT_ID = "6WXYLZ7CDN"  # Replace with your agent ID
AGENT_ALIAS_ID = "NGBVDSQOCG"  # Default test alias or your custom alias
AWS_REGION = "us-west-2"  # Replace with your preferred region

# Initialize Bedrock client
bedrock_agent_runtime = boto3.client(
    'bedrock-agent-runtime',
    region_name=AWS_REGION
)


In [10]:
def call_bedrock_agent(prompt: str, session_id: str = None) -> Dict[str, Any]:
    """
    Call a Bedrock agent with the given prompt.
    
    Args:
        prompt (str): The prompt/question to send to the agent
        session_id (str, optional): Session ID for conversation continuity
    
    Returns:
        Dict containing the agent's response and metadata
    """
    if session_id is None:
        session_id = f"session-{int(time.time())}"
    
    try:
        # Call the Bedrock agent
        response = bedrock_agent_runtime.invoke_agent(
            agentId=AGENT_ID,
            agentAliasId=AGENT_ALIAS_ID,
            sessionId=session_id,
            inputText=prompt
        )
        
        # Process the streaming response
        result = {
            "session_id": session_id,
            "response_text": "",
            "citations": [],
            "trace": [],
            "metadata": {}
        }
        
        for event in response['completion']:
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    text = chunk['bytes'].decode('utf-8')
                    result["response_text"] += text
                    
            elif 'trace' in event:
                result["trace"].append(event['trace'])
                
        return result
        
    except Exception as e:
        return {
            "error": str(e),
            "session_id": session_id
        }


In [11]:
# Example: Call the agent with a sample prompt
sample_prompt = "Hello! Can you help me understand what you can do?"

print("Calling Bedrock agent...")
print(f"Prompt: {sample_prompt}")
print("-" * 50)

# Make sure to update AGENT_ID above before running this cell
result = call_bedrock_agent(sample_prompt)

if "error" in result:
    print(f"Error: {result['error']}")
else:
    print(f"Session ID: {result['session_id']}")
    print(f"Response: {result['response_text']}")
    
    if result['trace']:
        print("\nTrace information:")
        for trace in result['trace']:
            print(f"  {trace}")


Calling Bedrock agent...
Prompt: Hello! Can you help me understand what you can do?
--------------------------------------------------
Session ID: session-1755575659
Response: Sorry I cannot answer


## Interactive Agent Call

Use the cell below to call your agent with any prompt you want to test:


In [17]:
# Call your agent with a custom prompt
your_prompt = "My todo is to take out the trash, it is high priority, and just a basic chore."

# Optional: specify a session ID to maintain conversation context
# session_id = "my-custom-session"  # Uncomment and set if needed

print(f"Sending prompt: {your_prompt}")
print("=" * 60)

result = call_bedrock_agent(your_prompt)
# result = call_bedrock_agent(your_prompt, session_id)  # With custom session

if "error" in result:
    print(f"❌ Error: {result['error']}")
    print("\nTroubleshooting tips:")
    print("1. Make sure your AGENT_ID is correctly set")
    print("2. Verify your AWS credentials are configured")
    print("3. Check that your agent is deployed and active")
    print("4. Ensure you have the necessary IAM permissions")
else:
    print(f"✅ Success!")
    print(f"Session ID: {result['session_id']}")
    print(f"\n🤖 Agent Response:")
    print(result['response_text'])
    
    if result['trace']:
        print(f"\n🔍 Trace Details:")
        for i, trace in enumerate(result['trace'], 1):
            print(f"  Step {i}: {trace}")


Sending prompt: My todo is to take out the trash, it is high priority, and just a basic chore.
✅ Success!
Session ID: session-1755576090

🤖 Agent Response:
Here is your todo list with the task you provided:
```json
[
  {
    "task": "take out the trash",
    "category": "basic chore",
    "notes": "",
    "priority": "high"
  }
]

